In [10]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.autograd import Variable
trunck_size = 1
device = torch.device("cpu")
np.random.seed(321)
torch.manual_seed(123) 


In [11]:
X_train = np.array(list(range(-150,150,1))) / 100
index = np.random.permutation(len(X_train))
X_train = X_train[index].reshape(1,-1)
Y_train = np.zeros((1,X_train.shape[1]))
for k in range(X_train.shape[1]):
    Y_train[0,k] = ((X_train[0,0:k+1].sum()))

X_test = np.array([-9,-10,13,-8,0,30,14,-6,7,8,-2,5]).reshape(1,-1) / 5
Y_test = np.zeros((1,X_test.shape[1]))
for k in range(X_test.shape[1]):
    Y_test[0,k] = ((X_test[0,0:k+1].sum()))

print(Y_test)

[[-1.8 -3.8 -1.2 -2.8 -2.8  3.2  6.   4.8  6.2  7.8  7.4  8.4]]


In [12]:
class LSTMCell(nn.Module):
    def __init__(self, flatten_dim, hidden_size, bias):
        super(LSTMCell, self).__init__()
    
        self.flatten_dim = flatten_dim
        self.hidden_size = hidden_size

        self.bias        = bias
        
        self.i2h = nn.Linear(flatten_dim, 4*hidden_size, bias=bias)
        self.h2h = nn.Linear(hidden_size, 4*hidden_size, bias=bias)
        
 
        
    def forward(self, x, cur_state):
        
        if cur_state is None:
            cur_state = (Variable(torch.zeros(1, self.hidden_size)).to(device).float(),
                Variable(torch.zeros(1, self.hidden_size)).to(device).float())
        
        x = x.view(1,-1)
        c_cur, h_cur = cur_state
        preact = self.i2h(x) + self.h2h(h_cur)
        #print(preact.size())
        ingate, forgetgate, cellgate, outgate = preact.chunk(4, 1)
        
        
        ingate = torch.sigmoid(ingate)
        forgetgate = torch.sigmoid(forgetgate)
        cellgate = torch.tanh(cellgate)
        outgate = torch.sigmoid(outgate)
        
        c_next = (forgetgate * c_cur) + (ingate * cellgate)
        h_next = outgate * torch.tanh(c_next)
        
        next_state = (c_next,h_next)
        
        #softmax_out = F.softmax(self.linear(h_next.view(1,-1)),dim=1)
        
        return next_state

class RNNCell(nn.Module):
    def __init__(self, flatten_dim, hidden_size, bias):
        super(RNNCell, self).__init__()
    
        self.flatten_dim = flatten_dim
        self.hidden_size = hidden_size

        self.bias        = bias
        
        self.i2h = nn.Linear(flatten_dim, hidden_size, bias=bias)
        self.h2h = nn.Linear(hidden_size, hidden_size, bias=bias)
        
    def forward(self,x,cur_h):
        
        if cur_h is None:
            cur_h = Variable(torch.zeros(1, self.hidden_size)).to(device).float()
        
        act = self.i2h(x) + self.h2h(cur_h)
        
        next_h = torch.tanh(act)
        
        return next_h
        

In [13]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__() 
        self.lstm1 = LSTMCell(1,1,True)
        #self.lstm2 = LSTMCell(4,4,True)
        self.lin1 = nn.Linear(in_features=1,out_features=1,bias=True)
        #self.rel = nn.Sigmoid()
        #self.drop = nn.Dropout(p=0.00)
        #self.lin2 = nn.Linear(in_features=1,out_features=1,bias=True)
        
    def forward(self,x,state1):
        c1,h1 = self.lstm1(x,state1)
        #c2,h2 = self.lstm2(h1,state2)
        out = self.lin1(h1)
        #r1 = self.rel(l1)
        #d1 = self.drop(r1)
        #out = self.lin2(d1)
        next_state1 = (c1,h1)
        #next_state2 = (c2,h2)
        return next_state1,out

class ModelR(nn.Module):
    def __init__(self):
        super(ModelR, self).__init__() 
        self.rnn1 = RNNCell(1,1,True)
        self.lin1 = nn.Linear(in_features=1,out_features=1,bias=True)

    def forward(self,x,state1):
        h1 = self.rnn1(x,state1)
        out = self.lin1(h1)
        
        next_state1 = h1
        return next_state1,out

In [14]:
model = Model().to(device).float()
optimizer = torch.optim.RMSprop(model.parameters(),lr=1.e-3)


# In[26]:


modelr = ModelR().to(device).float()
optimizerr = torch.optim.RMSprop(modelr.parameters(),lr=1.e-3)


In [15]:
def calculate_loss(out,real,mdl,tr_test):
    
    l1_reg = None
    l2_reg = None
    
    for W in mdl.parameters():
        if l1_reg is None:
            l1_reg = W.norm(1)
        else:
            l1_reg = l1_reg + W.norm(1)
            
        if l2_reg is None:
            l2_reg = W.norm(2)
        else:
            l2_reg = l2_reg + W.norm(2)
    
    l1_lambda = 0.00
    l2_lambda = 0.00
    if tr_test == "train":
        loss = ((out-real)**2).sum() + l2_lambda*l2_reg + l1_lambda*l1_reg
    if tr_test == "test" :
        loss = torch.sum((out-real)**2)
    return loss

In [16]:
def train(m,opt):
    m = m.train()
    #epoch_loss = 0.0
    out_list = []
    
    state1 = None
        
    for trunck in range(0,X_train.shape[1],trunck_size):
        X_trunck = X_train[:,trunck:trunck+trunck_size]
        Y_trunck = Y_train[:,trunck:trunck+trunck_size]
            
        """if state1 is not None:
            state1 = (Variable(state1[0].data),Variable(state1[1].data))"""
            
        if state1 is not None:
            state1 = Variable(state1.data)
                
        #state1 = None

        trunck_loss = 0.0
        for k in range(X_trunck.shape[1]):
            inp = X_trunck[:,k:k+1]
            real = Y_trunck[:,k:k+1]
            inp = torch.from_numpy(inp).to(device).float()
            real = torch.from_numpy(real).to(device).float()
            state1,out = m.forward(inp,state1)
            loss = calculate_loss(out,real,m,"train")
            trunck_loss += loss
            out_list.append(out.item())
            #epoch_loss += loss.item()
        opt.zero_grad()
        av_trunck_loss = trunck_loss / X_trunck.shape[1]
        av_trunck_loss.backward()
        opt.step()
            
       
    epoch_loss = (((np.array(out_list)-Y_train)**2).sum())    
    
    return epoch_loss,out_list

In [17]:
def test1(m):
    m = m.eval()
    out_list = []

    state1 = None
        
    for k in range(X_test.shape[1]):
        inp = X_test[:,k:k+1]
        real = Y_test[:,k:k+1]
        inp = torch.from_numpy(inp).to(device).float()
        real = torch.from_numpy(real).to(device).float()
        state1,out = m.forward(inp,state1)
        loss = calculate_loss(out,real,m,"test")
        out_list.append(out.item())
        
    epoch_loss = (((np.array(out_list)-Y_test)**2).sum())   
    
    return epoch_loss,out_list

def test2(m):
    m = m.eval()
    out_list = []

    state1 = None
        
    for k in range(X_train.shape[1]):
        inp = X_train[:,k:k+1]
        real = Y_train[:,k:k+1]
        inp = torch.from_numpy(inp).to(device).float()
        real = torch.from_numpy(real).to(device).float()
        state1,out = m.forward(inp,state1)
        loss = calculate_loss(out,real,m,"test")
        out_list.append(out.item())
    epoch_loss = (((np.array(out_list)-Y_train)**2).sum()) 
    return epoch_loss,out_list

In [ ]:
for epoch in range(1,8001,1):
    epoch_loss,out = train(modelr,optimizerr)
    test_loss,test_out = test1(modelr)
    if epoch % 400 == 0:
        print("epoch: "+str(epoch))
        print("train loss: "+str(epoch_loss))
        print("test loss: "+str(test_loss))
        print("***")


epoch: 400
train loss: 1473.17285401
test loss: 37.4767455048
***
epoch: 800
train loss: 937.201117927
test loss: 26.4614719602
***
epoch: 1200
train loss: 863.642201365
test loss: 41.1697898841
***


In [ ]:
test_loss,test_out = test1(modelr)


print(np.array(test_out))
print(Y_test)
print(test_loss)

In [ ]:
tr_loss,tr_out = test2(modelr)

print(np.array(tr_out))
print(Y_train)
print(tr_loss)



In [ ]:
((Y_train-tr_out)**2).sum()

In [ ]:
a = torch.ones(2,2) -10
a.sum()
